**<font size=4, color='#C5B358'>Data used-</font> web scraped data of every player playing in the *premier league 2017-2018*** and a private dataset BeerImage for Image mask (wordcloud visualiztion) 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 

from termcolor import colored

import seaborn as sns

from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 

from sklearn.preprocessing import LabelEncoder

from PIL import Image #for image mask
from subprocess import check_output
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)

import warnings
warnings.filterwarnings("ignore")
# Any results you write to the current directory are saved as output.

<font size=4, color='#C5B358'>Preparing & Sanitizing the Data</font>

In [ ]:
df = pd.read_json("../input/english-premier-league-player-data-20172018/fpl_data.json") #Data is of Players who played in 2017-18 Premier League (web scraped) 
df = df.T
df.reset_index(level=0, inplace = True)
df = df.rename(columns={"index": "Player Name"})
df['Goals']= df['Goals'].astype(int)
df['Goals']=df['Goals'].fillna(0)
df[df['Club']=='Sevilla'].dropna(inplace=True)
PlayerData = df.sort_values(by=['Goals'],ascending=False) #Data Sorted by Goals
tempPlayerData = PlayerData  #getting required data for graph
#tempPlayerData.head(10)      #top 10 row printed

#Sanitizing Data
df = df.fillna(0)
df['Accurate Long Balls'] = df['Accurate Long Balls'].str.replace(',', '')
df['Clearances'] = df['Clearances'].str.replace(',', '')
df['Crosses'] = df['Crosses'].str.replace(',', '')
df['Duels Lost'] = df['Duels Lost'].str.replace(',', '')
df['Duels Lost'] = df['Duels Lost'].str.replace(',', '')
df['Duels Won'] = df['Duels Won'].str.replace(',', '')
df['Goal Kicks'] = df['Goal Kicks'].str.replace(',', '')
df['Recoveries'] = df['Recoveries'].str.replace(',', '')
df['Passes'] = df['Passes'].str.replace(',', '')
df['Shooting Accuracy %'] = df['Shooting Accuracy %'].str.replace('%', '')
df['Cross Accuracy %'] = df['Cross Accuracy %'].str.replace('%', '')
df['Tackle Success %'] = df['Tackle Success %'].str.replace('%', '')
df["Passes Per Match"]= df["Passes Per Match"].astype(float)
df["Wins"] = df["Wins"].astype(int)
df["Shots On Target"] = df["Shots On Target"].astype(int)
df["Assists"] = df["Assists"].astype(int)
df["Penalties Saved"] = df["Penalties Saved"].astype(int)
df["Penalties Scored"] = df["Penalties Scored"].astype(int)
df["Saves"] = df["Saves"].astype(int)
df["Blocked Shots"] = df["Blocked Shots"].astype(int)

*<font size=4, color='#C5B358'>Let's watch our Favorite Club over a Pint of Beer </font>*

In [ ]:
df.to_csv('Football.csv', index = False)

In [ ]:
#word cloud - Popular Cities (top 50)
wave_mask3 = np.array(Image.open( "../input/beerimage/tulip-beer-glass.jpg"))
wordcloud = (WordCloud( max_words = 200,stopwords=stopwords, mask = wave_mask3, background_color='#361F1B',margin=10).generate_from_frequencies(df.groupby('Club')['Wins'].count()))
fig = plt.figure(1,figsize=(15, 15))
plt.imshow(wordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

*<font size=4, color='#C5B358'>How Average are Top 10 Clubs based on Goals scored by Players </font>*

In [ ]:
cm = sns.light_palette("#bc9b1b", as_cmap=True)
temp = df.groupby("Club")["Club","Goals"].sum().sort_values("Goals",ascending=False).head(10)
plotdata = df[df["Club"].isin(temp.index)]
print(colored("#Top Club & the Average stats",'blue'))
display(plotdata.groupby('Club')['Assists','Goals','Passes','Shots On Target','Tackles',"Penalties Scored","Saves"].mean().style.background_gradient(cmap=cm))

*<font size=4, color='#C5B358'>t-SNE visualization of Top 10 Clubs </font>*

In [ ]:
#Data Prep 
topclubs = df.groupby('Club')['Wins'].count().sort_values(ascending=False).head(10).index
Xt = df.iloc[:,1:-1]
Xt = Xt[Xt['Club'].isin(topclubs)]
#Xt = Xt[Xt.Club != ''].reset_index(drop=True)
#Xt = Xt[Xt.Club != 'Sevilla'].reset_index(drop=True)
#Xt = Xt.sort_values()
yt = Xt['Club']
Xt = Xt.drop(columns=['Club','Position','Team Play','Defence','Discipline','Successful /s', 'Successful 0/0s', 'Successful 5/5s',
       'Successful 50/50s'])


Xt['Crosses'] = Xt['Crosses'].str.replace(',', '')
Xt['Crosses'] = Xt['Crosses'].fillna(0)
Xt['Crosses'] = Xt['Crosses'].astype(int)
Xt['Accurate Long Balls'] = Xt['Accurate Long Balls'].fillna(0)
Xt['Accurate Long Balls'] = Xt['Accurate Long Balls'].astype(int)
Xt['Shots'] = Xt['Shots'].str.replace(',', '')
Xt['Shots']=Xt['Shots'].fillna(0)
Xt['Shots']=Xt['Shots'].astype(int)
Xt['Tackles'] = Xt['Tackles'].str.replace(',', '')
Xt['Tackles']=Xt['Tackles'].fillna(0)
Xt['Tackles']=Xt['Tackles'].astype(int)
Xt['Aerial Battles Lost'] = Xt['Aerial Battles Lost'].fillna(0)
Xt['Aerial Battles Lost'] = Xt['Aerial Battles Lost'].astype(int)
Xt['Aerial Battles Won'] = Xt['Aerial Battles Won'].fillna(0)
Xt['Aerial Battles Won'] = Xt['Aerial Battles Won'].astype(int)
Xt['Big Chances Created'] = Xt['Big Chances Created'].astype(int)
Xt['Headed Clearance'] = Xt['Headed Clearance'].str.replace(',', '')
Xt['Headed Clearance'] = Xt['Headed Clearance'].fillna(0)
Xt['Headed Clearance'] = Xt['Headed Clearance'].astype(int)
Xt['Errors Leading To Goal']= Xt['Errors Leading To Goal'].astype(int)
Xt['Goal Kicks']= Xt['Goal Kicks'].fillna(0)
Xt['Goal Kicks']= Xt['Goal Kicks'].astype(int)
Xt['Goalkeeping']= Xt['Goalkeeping'].str.replace('','0')
Xt['Goalkeeping']= Xt['Goalkeeping'].fillna(0)
Xt['Goalkeeping']= Xt['Goalkeeping'].astype(int)
Xt['Goals Conceded']= Xt['Goals Conceded'].astype(int)
Xt['Goals From Freekick']= Xt['Goals From Freekick'].astype(int)
Xt['Goals Per Match']= Xt['Goals Per Match'].astype(float)
Xt['Goals With Header']= Xt['Goals With Header'].astype(int)
Xt['Goals With Left Foot']= Xt['Goals With Left Foot'].astype(int)
Xt['Goals With Right Foot']= Xt['Goals With Right Foot'].astype(int)
Xt['Hit Woodwork']= Xt['Hit Woodwork'].astype(int)
Xt['Interceptions']= Xt['Interceptions'].astype(int)
Xt['Last Man Tackles']= Xt['Last Man Tackles'].astype(int)
Xt['Punches']= Xt['Punches'].astype(int)
Xt['Offsides']= Xt['Offsides'].astype(int)
Xt['Through Balls']= Xt['Through Balls'].astype(int)
Xt['Throw Outs']= Xt['Throw Outs'].str.replace(',','')
Xt['Throw Outs']= Xt['Throw Outs'].fillna(0)
Xt['Throw Outs']= Xt['Throw Outs'].astype(int)
Xt['Sweeper Clearances']= Xt['Sweeper Clearances'].astype(int)
Xt['Own Goals']= Xt['Own Goals'].astype(int)
Xt['Red Cards']= Xt['Red Cards'].astype(int)
Xt['Appearances']= Xt['Appearances'].astype(int)
Xt['Attack']= Xt['Attack'].str.replace('','0')
Xt['Attack']= Xt['Attack'].fillna(0)
Xt['Attack']= Xt['Attack'].astype(int)
Xt['Big Chances Missed']= Xt['Big Chances Missed'].astype(int)
Xt['Catches']= Xt['Catches'].astype(int)
Xt['Clean Sheets']= Xt['Clean Sheets'].astype(int)
Xt['Clearances']= Xt['Clearances'].fillna(0)
Xt['Clearances']= Xt['Clearances'].astype(int)
Xt['Clearances Off Line']= Xt['Clearances Off Line'].astype(int)
Xt['Cross Accuracy %']= Xt['Cross Accuracy %'].fillna(0)
Xt['Cross Accuracy %']= Xt['Cross Accuracy %'].astype(int)
Xt['Duels Lost']= Xt['Duels Lost'].fillna(0)
Xt['Duels Lost']= Xt['Duels Lost'].astype(int)
Xt['Duels Won']= Xt['Duels Won'].fillna(0)
Xt['Duels Won']= Xt['Duels Won'].astype(int)
Xt['Fouls']= Xt['Fouls'].astype(int)
Xt['High Claims']= Xt['High Claims'].astype(int)
Xt['Losses']= Xt['Losses'].astype(int)
Xt['Passes']= Xt['Passes'].astype(int)
Xt['Recoveries']= Xt['Recoveries'].fillna(0)
Xt['Recoveries']= Xt['Recoveries'].astype(int)
Xt['Shooting Accuracy %']= Xt['Shooting Accuracy %'].astype(float)
Xt['Tackle Success %']= Xt['Tackle Success %'].astype(float)
Xt = Xt.fillna(0)

#encoding labels
lb = LabelEncoder()
y = pd.DataFrame(yt)  
y['Label'] = lb.fit_transform(y['Club'])

In [ ]:
# Utility function to visualize the outputs of  t-SNE

def v_scatter(x, c,name,label):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(c[label]))
    palette = np.array(sns.color_palette("hls", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(15, 15))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[c['Label'].astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit corresponding to the label
    txts = []

    for i in range(num_classes):

        # Position of each label at median of data points.
        xtext, ytext = np.median(x[c[label] == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(c[c[label] == i][name].unique()), fontsize=10)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [ ]:
import matplotlib.patheffects as PathEffects
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
                rc={"lines.linewidth": 2.5})
RS = 123

from sklearn.manifold import TSNE
v_tsne = TSNE(random_state=RS).fit_transform(Xt)
v_scatter(v_tsne, y,'Club','Label')


*<font size=4, color='#C5B358'>PCA visualization of Top 10 Clubs </font>*

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
pca_result = pca.fit_transform(Xt)
pca_df = pd.DataFrame(columns = ['pca1','pca2','pca3','pca4'])

pca_df['pca1'] = pca_result[:,0]
pca_df['pca2'] = pca_result[:,1]
pca_df['pca3'] = pca_result[:,2]
pca_df['pca4'] = pca_result[:,3]

print('Variance explained per principal component: {}'.format(pca.explained_variance_ratio_))

top_two_comp = pca_df[['pca1','pca2']] # taking first and second principal component

v_scatter(top_two_comp.values,y,'Club','Label') # Visualizing the PCA output

In [ ]:
#function for the 3D Scatter plot
def plot3D(xlabel, ylabel, n,desc='Player Name'):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    fig.subplots_adjust(left=0.4, right=2,bottom=2,top=4)
    
    tempPlayerData[[xlabel,ylabel]] = tempPlayerData[[xlabel,ylabel]].fillna(0)
    
    z = np.array(tempPlayerData['Goals'])
    y = np.array(tempPlayerData[ylabel].astype(int))
    x = np.array(tempPlayerData[xlabel].astype(int))

    Desc = np.array(tempPlayerData[desc])
    
    a = 'right'
    for i in range(n):
        ax.scatter(x[i], y[i], z[i], c='orange', marker='*',s=100)
        ax.text(x[i], y[i], z[i], '%s'%(Desc[i]), color='r',alpha=0.8, fontsize=8,horizontalalignment=a,verticalalignment='bottom', \
           bbox=dict(facecolor='red', alpha=0.12)) 
        if a=='right':
            a = 'left'
        else:
            a = 'right'
        
    plt.title("Top %d Players"%(n))
    ax.set_zlabel('Goals Scored')
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    ax.w_xaxis.set_pane_color((1.0, 0.99,1.0,  1.0))
    ax.w_yaxis.set_pane_color((1.0, 0.99, 1.0, 1.0))
    ax.w_zaxis.set_pane_color((0.2, 1.0, 0.2, 1.0))
    #ax.mouse_init(rotate_btn=1, zoom_btn=3)
    plt.show()  

*<font size=4, color='#C5B358'>Some great players based on Goals, Tackles and shots on Target </font>* ***(Top 10)***

In [ ]:
plot3D('Shots On Target','Tackles',10)

*<font size=4, color='#C5B358'>Displaying Clubs instead of the Players' Name for above Plot</font>* ***(Top 5)***

In [ ]:
plot3D('Shots On Target','Tackles',5,"Club") #Uhhh Everton had/have the player with max Goals scored - Wayne Rooney

In [ ]:
#function for player comparision 
def PlayerCompare1D(x,y,desc):
    P1 = df[df["Player Name"]==x][desc].values.astype(str)
    P2 = df[df["Player Name"]==y][desc].values.astype(str)
    print("Player %s vs %s -- %s"%(x,y,desc))
    if P1[0]>P2[0]:
        print(colored("          %s"%(P1[0]),'green'),colored("          %s"%(P2[0]),'red'))
    else:
        print(colored("          %s"%(P1[0]),'red'),colored("          %s"%(P2[0]),'green'))
        


*<font size=4, color='#C5B358'>Comparing Two Players based on Shooting Accuracy</font>*

In [ ]:
PlayerCompare1D("Romelu Lukaku","Marcus Rashford","Shooting Accuracy %")

In [ ]:
PlayerCompare1D("Wayne Rooney","Paul Pogba","Shooting Accuracy %")

*<font size=4, color='#C5B358'>Player SpotLight - How well he Performed?</font>*

In [ ]:
#functions to get player power
def Playwell(trace1):
    x = df[df["Player Name"] == trace1]
    trace1 = go.Scatterpolar(
      r = [x['Assists'].values[0],x['Goals'].values[0],x['Passes Per Match'].values[0],x['Shots On Target'].values[0],x['Tackles'].values[0],x["Penalties Scored"].values[0]],
      theta = ['Assists','Goals','Passes Per Match','Shots On Target','Tackles',"Penalties Scored"],
      fill = 'toself',
      name = trace1
    )
    return trace1

def PlayerSpotlight(trace1):
    trace1 = Playwell(trace1)
    layout = go.Layout(
          xaxis=dict(
            domain=[0, 0.45]
            ),
            yaxis=dict(
            domain=[0, 0.45]
            ),
        
           
          showlegend = True,
          title = "Player Performance"
    )

    data = [trace1]
    fig = go.Figure(data=data, layout=layout)

    iplot(fig, filename = "Player stats")

In [ ]:
PlayerSpotlight("Wayne Rooney")

*<font size=4, color='#C5B358'>Search for a great Player (the likes of Rooney) Continues..</font>*

In [ ]:
#Function to compare the Pok¨¦mon powers
def PlayerFight(trace1,trace2,trace3,trace4):
    trace1 = Playwell(trace1) #calling the Helper function defined before
    trace2 = Playwell(trace2)
    trace3 = Playwell(trace3)
    trace4 = Playwell(trace4)
    layout = go.Layout(
      xaxis=dict(
            domain=[0, 0.50]
        ),
        yaxis=dict(
            domain=[0, 0.50]
        ),
        xaxis2=dict(
            domain=[0.70, 1]
        ),
        xaxis3=dict(
            domain=[0, 0.50],
            anchor='y3'
        ),
        xaxis4=dict(
            domain=[0.70, 1],
            anchor='y4'
        ),
        yaxis2=dict(
            domain=[0, 0.50],
            anchor='x2'
        ),
        yaxis3=dict(
            domain=[0.70, 1]
        ),
        yaxis4=dict(
            domain=[0.70, 1],
            anchor='x4'
        ),
      showlegend = True,
      title = "Player Fight (who will you bet on!)"
    )

    data = [trace1, trace2, trace3, trace4]
    fig = go.Figure(data=data, layout=layout)

    iplot(fig, filename = "Player Fight")

In [ ]:
PlayerFight("Romelu Lukaku","Marcus Rashford","Paul Pogba","Eden Hazard")

*Hazard and Lukaku* outperform others.*Lukaku* has good number of Goals & Shots on Target while *Hazard* has a pretty good spread of Capabilities! [This Plot can be enhanced for specific roles -> Midfielder, Striker, Defender & Goal Keeper separately. 

In [ ]:
#Getting Data specific to Man Utd.
ManUtdPlayerData = df[df["Club"]=="Manchester United"].sort_values(by="Position")
ManUtdPlayerData= ManUtdPlayerData.reset_index(drop=True)
#ManUtdPlayerData

In [ ]:
#Main hardcoded funtion for finding best Players, what we get as Best player is based on the parameters defined in the function and weights given to them
def scoreEvaluator(tempData, Pos):
    score = 0.0
    #i have tried to keep the weights simple 
    if Pos == "Goalkeeper":
        score = score - 1*float(tempData["Errors Leading To Goal"]) \
            + 0.5*float(tempData["Assists"]) \
            - 1*float(tempData["Red Cards"]) \
            + 1.0*float(tempData["Penalties Saved"]) - 0.9*float(tempData["Fouls"]) \
            + 0.2*float(tempData["Appearances"]) - 0.8*float(tempData["Own Goals"]) \
            + 0.8*float(tempData["Blocked Shots"]) - 1*float(tempData["Goals Conceded"]) + \
            0.5*float(tempData["Accurate Long Balls"]) + 3*float(tempData["Saves"])
        return score
    
    if Pos == "Defender":
        score = score + 0.5*float(tempData["Goals"]) - 1*float(tempData["Errors Leading To Goal"]) \
            + 0.9*float(tempData["Shots On Target"]) + 0.5*float(tempData["Assists"])  \
            + 0.5*float(tempData["Tackles"]) - 1*float(tempData["Red Cards"]) \
            + 1.0*float(tempData["Penalties Saved"]) - 0.9*float(tempData["Fouls"]) \
            + 0.8*float(tempData["Goals With Header"]) - 0.8*float(tempData["Own Goals"]) \
            + 0.8*float(tempData["Blocked Shots"]) + 0.5*float(tempData["Passes Per Match"])
        return score
    
    if Pos =="Midfielder":
        score = score + 0.5*float(tempData["Shooting Accuracy %"]) +0.8*float(tempData["Goals"]) \
            + 0.9*float(tempData["Shots On Target"]) + 0.9*float(tempData["Assists"]) \
            + 0.9*float(tempData["Tackle Success %"]) - 1*float(tempData["Red Cards"]) \
            + 0.8*float(tempData["Penalties Scored"]) - 0.9*float(tempData["Fouls"]) \
            + 0.5*float(tempData["Goals With Header"]) - 0.8*float(tempData["Own Goals"]) \
            + 0.5*float(tempData["Goals From Freekick"]) + 1.0*float(tempData["Passes Per Match"])
        return score
    
    if Pos == "Forward":  
        score = score + 0.8*float(tempData["Shooting Accuracy %"]) +1*float(tempData["Goals"]) \
            + 0.9*float(tempData["Shots On Target"]) + 0.5*float(tempData["Assists"]) \
            + 0.5*float(tempData["Tackle Success %"]) - 1*float(tempData["Red Cards"]) \
            + 0.8*float(tempData["Penalties Scored"]) - 0.9*float(tempData["Fouls"]) \
            + 0.8*float(tempData["Goals With Header"]) - 0.8*float(tempData["Own Goals"]) \
            + 0.8*float(tempData["Goals From Freekick"]) + 0.5*float(tempData["Passes Per Match"]) \
            + 0.8*float(tempData["Cross Accuracy %"])
        return score




def BestUtdPlayer(Pos):
    if Pos not in ["Defender","Forward","Midfielder","Goalkeeper"]:
        print("Invalid Position given - ReTry")
        return
    tempData = ManUtdPlayerData[ManUtdPlayerData["Position"]==Pos]
    nRows = tempData.shape[0]
    MaxScore = 0.0 
    j = 0
    tempData["Score"] = pd.Series([1.0]*nRows)  #initializing a score of 1 for each row
    for i in range(nRows):
        tempData["Score"].iloc[i] = tempData["Score"].iloc[i] + scoreEvaluator(tempData.iloc[i],Pos)
        if tempData["Score"].iloc[i] > MaxScore:
            MaxScore = tempData["Score"].iloc[i]
            j = i 
    print(colored("Best Utd %s is %s"%(Pos,tempData["Player Name"].iloc[j]),'blue'))
    

*<font size=4, color='#C5B358'>Best Manchester United Players Based on our Analysis (**may not be your choice**)</font>*

In [ ]:
BestUtdPlayer("Goalkeeper")
BestUtdPlayer("Defender")
BestUtdPlayer("Midfielder")
BestUtdPlayer("Forward")


In [ ]:
ManUtdPlayerData.to_csv('ManUtdPlayerData.csv', index = False)

*<font size=4, color='#C5B358'>Regression Plot to see what ability correlate more with Win!</font>*

In [ ]:
#Plotting
fig, (ax) = plt.subplots(figsize=(12,9),ncols=2, nrows=3)

print("Passes Per Match Correlation to Wins",colored(df["Wins"].corr(df["Passes Per Match"]),'green')) #Pearson Correlation Coef.
sns.regplot(x="Wins",y="Passes Per Match",data=df,ax=ax[0][0])

print("Shots On Target Correlation to Wins ",colored(df["Wins"].corr(df["Shots On Target"]),'green'))
sns.regplot(x="Wins",y="Shots On Target",data=df,ax=ax[0][1])

print("Assists Correlation to Wins         ",colored(df["Wins"].corr(df["Assists"]),'green')) 
sns.regplot(x="Wins",y="Assists",data=df,ax=ax[1][0])

print("Penalites Scored Correlation to Wins",colored(df["Wins"].corr(df["Penalties Scored"]),'green'))
sns.regplot(x="Wins",y="Penalties Scored",data=df,ax=ax[1][1])

print("Goals Correlation to Wins           ",colored(df["Wins"].corr(df["Goals"]),'green'))
sns.regplot(x="Wins",y="Goals",data=df,ax=ax[2][0])

print("Saves Correlation to Wins           ",colored(df["Wins"].corr(df["Saves"]),'green'))
sns.regplot(x="Wins",y="Saves",data=df,ax=ax[2][1])

plt.show()

**Goals, Passes and Assists are more correlated to Wins then Shots on Targets, Saves or Penalties Saved! (as shown by above plots). Also Assists correlate more to Wins then does Goals**

*<font size=4, color='#C5B358'>3D Scatter plot of Goals, Passes Per Match and Wins!</font>*

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np


fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(111, projection='3d')

n = 100

# Make data.
xs = df["Goals"]
ys = df["Passes Per Match"]
zs = df["Wins"]


ax.scatter(xs, ys, zs,  marker='*',color='green')
ax.set_xlabel('Goals')
ax.set_ylabel('Passes Per Match')
ax.set_zlabel('Wins')

plt.show()


*<font size=4, color='#C5B358'>Can Passes help Goal?</font>*

In [ ]:
sns.jointplot(x='Passes Per Match',y='Goals',data=df,kind='kde',color='blue')
plt.show()

Doesn't seems to be correlated

*<font size=4, color='#C5B358'>Top 10 Clubs by Maximum Goals Scored!</font>*

In [ ]:
MG = df.groupby("Club")["Goals"].sum().sort_values(ascending=False).head(10)
fig = plt.figure(figsize=(15,6))
plt.bar(MG.index,MG.values,color='green')                                #plotting
plt.xlabel("Clubs",color='green')
plt.ylabel("Total Goals (in Premiere League)",color='green')
plt.show()

*<font size=4, color='#C5B358'>Top 10 Clubs by Maximum Wins!</font>*

In [ ]:
MW = df.groupby("Club")["Wins"].sum().sort_values(ascending=False).head(10)
fig = plt.figure(figsize=(15,6))
plt.bar(MW.index,MW.values)                              #plotting
plt.xlabel("Clubs",color='blue')
plt.ylabel("Total Wins (in Premiere League)",color='blue')
plt.show()

*<font size=4, color='#C5B358'>Top Defensive Clubs</font>*

In [ ]:
cm = sns.light_palette("yellow", as_cmap=True)
df.groupby('Club')['Blocked Shots','Penalties Saved','Saves'].sum().sort_values(by='Saves',ascending = False).head(20).style.background_gradient(cmap=cm)

*<font size=4, color='#C5B358'>Top Aggressive Clubs</font>*

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
df.groupby('Club')['Goals','Penalties Scored','Assists'].sum().sort_values(by='Goals',ascending = False).head(20).style.background_gradient(cmap=cm)

*<font size=4, color='#C5B358'>How Bad!, can one play?</font>*

In [ ]:
fig = plt.figure(figsize=(12,6))
print(colored("Foul Play Distributions",'blue'))
plt.subplot(1,3,1)
sns.distplot(df["Red Cards"].astype(int),color='red')
plt.subplot(1,3,2)
sns.distplot(df["Yellow Cards"].astype(int),color='yellow')
plt.subplot(1,3,3)
sns.distplot(df["Fouls"].astype(int),color='grey')

plt.show()

While Yellow Cards and Fouls have close to normal distribution. Red Cards are wierdly Multimodial!

*<font size=4, color='#C5B358'>Does having a good shooting accuracy means you can score more goals?</font>*

In [ ]:
#Data Prep
dfc = df.sort_values(by='Goals',ascending=False).head(10) #Looking at top 10 Goal Scorer of 17-18 
dfc['Shooting Accuracy %'] = dfc['Shooting Accuracy %'].astype(float)

#plotting
plt.style.use('dark_background')
fig = plt.figure(figsize=(12,9))
a= dfc.groupby("Goals")['Shooting Accuracy %'].mean().index
b = dfc.groupby("Goals")['Shooting Accuracy %'].mean().values
plt.plot(a,b,ls='dashed')
plt.ylabel('Shooting Accuracy %', color = 'red')
plt.xlabel("Goals", color = 'red')
plt.title("Top 10 Strikers (in terms of #Goals)",color='green')
plt.show()

*Result Inconclusive,* though the range of shooting Accuracy is narrow between ~38 to 46% but the higher the number of Goals doesn't seem to have a strong relationship with number of Goals


<br></br>
*<font size=4, color='#C5B358'>If not shooting accuracy what makes a Striker top-notch?</font>*

In [ ]:
#Data Prep, using same data as above dfc (#top 10 Strikers in terms of Goals)
dfc['Tackles'] = dfc['Tackles'].astype(int)
dfc['Assists'] = dfc['Assists'].astype(int)
dfc['Passes Per Match'] = dfc['Passes Per Match'].astype(int)
dfc['Shots On Target'] = dfc['Shots On Target'].astype(int)
dfc['Penalties Scored'] = dfc['Penalties Scored'].astype(int)

#plotting

fig = plt.figure(figsize=(12,9))
def plotline(i): #main plotting function
    a= dfc.groupby("Goals")[i].mean().index
    b = dfc.groupby("Goals")[i].mean().values
    plt.plot(a,b,ls='dashed')
cap = ['Tackles','Assists','Passes Per Match','Shots On Target','Penalties Scored'] #capabilities we looking for
for i in cap:
    plotline(i)
plt.legend(cap, loc='upper left')
plt.ylabel('Total #', color = 'red')
plt.xlabel("Goals", color = 'red')
plt.title("Top 10 Strikers (in terms of #Goals)",color='green')
plt.show()

*Shots on Target* and *Tackles* seems to be the top two Capabilites that makes you a High Goal Scorer

<br></br>
*<font size=4, color='#C5B358'>Best come-back Clubs!!!</font>*

In [ ]:
#Data Prep
df[['Aerial Battles Won','Duels Won','Recoveries','Tackle Success %']]= df[['Aerial Battles Won','Duels Won','Recoveries','Tackle Success %']].fillna(0).astype(int)
#Plotting
cm = sns.light_palette("orange", as_cmap=True)
df.groupby("Club")['Aerial Battles Won','Duels Won','Tackle Success %','Recoveries'].sum().sort_values(by='Recoveries',ascending=False).head(20).style.background_gradient(cmap=cm)

<br></br>
*<font size=4, color='#C5B358'>Every Team loves taking chances, but what creates Big Chances?</font>*

In [ ]:
#Data Prep
AB = df.copy()
AB['Accurate Long Balls']= AB['Accurate Long Balls'].fillna(0)
AB['Big Chances Created']=AB['Big Chances Created'].fillna(0)
AB['Crosses'] = AB['Crosses'].fillna(0)
AB['Crosses'] = AB['Crosses'].astype(int) #making sure all required fields are integers
AB['Big Chances Created'] = AB['Big Chances Created'].astype(int)
AB['Accurate Long Balls'] = AB['Accurate Long Balls'].astype(int)
AB['Tackles'] = AB['Tackles'].astype(int)
AB['Assists'] = AB['Assists'].astype(int)

#plotting
fig, (ax) = plt.subplots(figsize=(12,9),ncols=2, nrows=2,sharey=True)

print("Big Chances created correlation with Accurate long Balls",colored(AB['Big Chances Created'].corr(AB['Accurate Long Balls']),'green')) #Pearson Correlation Coef.
sns.regplot(x="Accurate Long Balls",y="Big Chances Created",data=AB,ax=ax[0][0])
print("Big Chances created correlation with Crosses ",colored(AB['Big Chances Created'].corr(AB['Crosses']),'green'))
sns.regplot(x="Crosses",y="Big Chances Created",data=AB,ax=ax[0][1])
print("Big Chances created correlation with Tackles",colored(AB['Big Chances Created'].corr(AB['Tackles']),'green')) #Pearson Correlation Coef.
sns.regplot(x="Tackles",y="Big Chances Created",data=AB,ax=ax[1][0])
print("Big Chances created correlation with Assists ",colored(AB['Big Chances Created'].corr(AB['Assists']),'green'))
sns.regplot(x="Assists",y="Big Chances Created",data=AB,ax=ax[1][1])
plt.show()

*Assists and Crosses* seems to be highly correlated with Big Chances Created. So, Players good in Assists and Crosses must be the ones who creating Big Chances for their Club! Lets look at them.

<br></br>
*<font size=4, color='#C5B358'>Big Chances, Assists, Crosses and the Players we love!</font>*

In [ ]:
cm = sns.light_palette("blue", as_cmap=True)
AB.groupby('Player Name')['Big Chances Created','Crosses','Assists'].sum().sort_values(by='Big Chances Created',ascending = False).head(20).style.background_gradient(cmap=cm)

^^Here we have the likes of *Wayne Rooney, David Silva*...

*<font size=4, color='#C5B358'>Top 3 Premier League Club- How well the saved the day?</font>*

In [ ]:
#Data Prep
dfg = df.groupby('Club')['Saves','Wins'].sum().sort_values(by='Wins',ascending=False).head(3)
cap = ['Saves','Wins'] #club names for legend
#plotting
fig = plt.figure(figsize=(12,9))
x=np.array([1.0,2.0,3.0])
ax = plt.subplot(111)
ax.bar(x-0.1,dfg['Saves'].values ,width=0.2,color='b',align='center')
ax.bar(x+0.1, dfg['Wins'].values,width=0.2,color='r',align='center')
LABELS = dfg.index
plt.legend(cap, loc='upper left')
plt.xticks(x, LABELS)
plt.ylabel('Total #', color = 'g')
plt.xlabel("Clubs", color = 'g')
plt.title("Top 3 Clubs (in terms of #Wins)",color='grey')
plt.show()


*<font size=4, color='#C5B358'>Box-Plot showcasing Player Capabilities - how average are they?</font>*

In [ ]:
sns.boxplot(data=df[['Assists','Crosses','Goals','Passes Per Match','Shots On Target','Tackles',"Penalties Scored"]], orient="h", palette="Set2")
plt.show()

Well Outliers is the name of Good Soccer, probably becuase we didn't segregate Players based on their role.

*<font size=4, color='#C5B358'>Does  good Shooting Accuracy leads to more Shots on Target?</font>*

In [ ]:
#Data Prep - removed all zero and NaN values
df['Shooting Accuracy %'] = df['Shooting Accuracy %'].fillna(0)
df['Shooting Accuracy %'] = df['Shooting Accuracy %'].astype(int)
df['Shots On Target'] = df['Shots On Target'].astype(int)
SD = df[df['Shooting Accuracy %']!=0]
SD = SD[SD['Shots On Target']!=0]
#plotting
sns.jointplot(x='Shooting Accuracy %',y='Shots On Target',data=SD, color='#6e4e37',xlim=(10,100),kind ='hex', 
              size = 8.0)
plt.show()

There is hardly any Correlation between *Shooting Accuracy* and *Shots on Target*. *Shooting Accuracy* is normally distributed with maximum *Shots on Target* value with Accuracy around 40%
<br></br>

*<font size=4, color='#C5B358'>Does Tackles Accuracy helps in winning Duels or Recovering?</font>*

In [ ]:
#data prep
tempD = df.dropna(subset=['Tackle Success %'])
tempD["Duels Won"] = tempD["Duels Won"].fillna(0)
tempD["Recoveries"] = tempD["Recoveries"].fillna(0)
tempD["Duels Won"] = tempD["Duels Won"].astype(int)
tempD["Recoveries"] = tempD["Recoveries"].astype(int)
tempD['Tackle Success %'] = tempD['Tackle Success %'].astype(int)
#plotting
fi = plt.figure(figsize=(12,9))
print("Tackle is Success, did we win the ball?")
plt.subplot(1,2,1)
plt.plot(tempD["Recoveries"],tempD["Tackle Success %"],'*',color='brown')
plt.xlabel('Recoveries')
plt.ylabel('Tackle Success %')
plt.subplot(1,2,2)
plt.plot(tempD["Duels Won"],tempD["Tackle Success %"],'*', color='green')
plt.xlabel('Duels Won')


plt.show()

*Tackle Success* doesn't seem to have an impact on *Recovery or Duels* winning; rather Duels Won and Recoveries by Players have a nice Normal Distribution. 
<br></br><br></br>

<br></br>
<font color='red'>Disclaimer:</font> The comments  made are personal opinion based on the Analysis and may not be 100% Accurate
<br></br>
**Thanks for Upvotes,  your Feedback as below will be highly appreciated :)**